In [5]:
import pandas as pd
ori_data = pd.read_csv('./telemetry/iot_telemetry_data.csv',sep = ',')

In [13]:
def device_map(device):
    if(device == 'b8:27:eb:bf:9d:51'): return 0
    elif(device == '00:0f:00:70:91:0a'): return 1
    else: return 2
def light_map(light):
    if(light == False): return 0
    else: return 1

def motion_map(motion):
    if(motion == False): return 0
    else: return 1

In [7]:
data = ori_data.copy()
data['device_mp'] =ori_data.apply(lambda x: device_map(x.device), axis = 1)
data['light_mp'] =ori_data.apply(lambda x: light_map(x.light), axis = 1)
data['motion_mp'] =ori_data.apply(lambda x: motion_map(x.motion), axis = 1)
data = data.drop(['device','light','motion'], axis = 1)

In [74]:
co_min = data['co'].min()
co_max = data['co'].max()
co_avg = data['co'].mean()
co_med = data['co'].median()
co_all = float(data['co'].mode())

In [75]:
box = [0,co_all,co_avg,co_med,co_max]

In [76]:
cut = pd.cut(data['co'],box)
co_lb = cut.values.codes
data['co_label'] = co_lb

In [102]:
import tensorflow as tf
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
from keras.models import *
from keras.callbacks import *
from keras.initializers import he_uniform, glorot_uniform, lecun_uniform
from keras.utils import to_categorical
from keras import backend as K
from keras import *
from keras.activations import *
from keras.utils import plot_model, multi_gpu_model, model_to_dot
from IPython.display import SVG
from keras_tqdm import TQDMNotebookCallback, TQDMCallback

from sklearn.model_selection import train_test_split
# from thundersvm import SVC, NuSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold
import bhtsne
from scipy.spatial.distance import pdist
from itertools import *
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from IPython.display import display
from scipy import sparse as sp
import scipy
from sklearn.metrics import *
from sklearn.preprocessing import *
from tqdm import tqdm, tqdm_pandas, tqdm_notebook, tqdm_gui
import os
import time
import random
import math
import igraph as ig
import networkx as nx
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline


In [103]:
from sklearn.cluster import KMeans,AgglomerativeClustering
from sklearn.metrics import normalized_mutual_info_score,adjusted_rand_score
num_classes = data['co_label'].map(lambda a:a).nunique()
label = data['co_label']

cluster_emb = data.copy()
cluster_emb['tab'] = range(len(cluster_emb))
# cluster_emb = cluster_emb.merge(label,how='left',on='nodes')
cluster_label = cluster_emb[['tab','co_label']]
cluster_emb = cluster_emb.sort_values(by=['tab']).drop(['tab','co_label'],axis=1).values
clf = KMeans(n_clusters=num_classes,n_init=20,n_jobs=-1,)
clf.fit(cluster_emb)
cluster_label['pred'] = clf.labels_

ori_label = list(range(num_classes))
max_score = {
    'acc' : [],
    'nmi' : [],
    'ars' : [],
}

def calc_cluster(i):
    dic = pd.DataFrame()
    dic['pred'] = ori_label
    dic['label'] = i
    dic = dict(dic.values)
    now = cluster_label['pred'].replace(dic).values
    return accuracy_score(lb,now), normalized_mutual_info_score(lb,now), adjusted_rand_score(lb,now)

p = [i for i in list(permutations(ori_label)) if i not in [ori_label]] 
lb = cluster_label['co_label'].values
res = Parallel(n_jobs=10,backend='loky',batch_size=256)(delayed(calc_cluster)(x) for x in tqdm_notebook(p))
for r in res:
    max_score['acc'].append(r[0])
    max_score['nmi'].append(r[1])
    max_score['ars'].append(r[2])
    
print("NMI {} \nARS {}\nACC {}".format(np.max(max_score['nmi']),
                                       np.max(max_score['ars']),
                                       np.max(max_score['acc'])))



NMI 0.02967991782461792 
ARS 0.013002882455742975
ACC 0.3092348167746012
